In [ ]:

{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import html\n",
    "from pyspark.sql import functions as f\n",
    "from pyspark.sql.functions import udf\n",
    "from pyspark.sql import SparkSession\n",
    "from pyspark.sql import DataFrame\n",
    "\n",
    "spark = SparkSession.builder.getOrCreate()\n",
    "\n",
    "RAW_PATH = \"/home/jovyan/data-sets/sentiment-140-training-data/RAW\"\n",
    "CLEAN_PATH = \"/home/jovyan/data-sets/sentiment-140-training-data/CLEAN\""
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Findings from our Data Exploration step:\n",
    "\n",
    "Re-ordered in the order in which we will pick these up:\n",
    "\n",
    "- __Problem:__ We need to apply a proper schema\n",
    "    > __Action:__ create `spark_reader` by applying `.schema()` to `SparkSession` object with proper schema DDL defined\n",
    "\n",
    "- __Problem:__ Encoding seems to be 'broken' - example: \n",
    "```markdown\n",
    "�����ߧ�ǿ�����ж�؜��� &lt;&lt;----I DID NOT KNOW I CUD or HOW TO DO ALL DAT ON MY PHONE TIL NOW. WOW..MY LIFE IS NOW COMPLETE. JK.\n",
    "```\n",
    "    > __Action:__ remove broken chars. Or, more rigorous, replace any none letter with a blank space << what happens when we do this?\n",
    "\n",
    "- __Problem:__ The date column needs fixing\n",
    "    > __Action:__ set `timestampFormat` option while reading csv\n",
    "\n",
    "- We need to extract twitter user names/handles (we'll extract it and call the output column users_mentioned)\n",
    "- We need to extract hashtags and replace them with the words from the hashtag (we'll extract it and call the output column hashtags)\n",
    "- We need to extract URLs, as our algorithm won't need that or use that (we'll simply remove it from the data)\n",
    "- The same goes for email-address\n",
    "- HTML does not appear properly unescaped, we're going to have to fix that (example: &lt;3 and s&amp;^t)\n",
    "\n",
    "\n",
    "...\n",
    "\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Have a first look at the data\n",
    "raw_data = spark.read.csv(PATH)  # .cache()\n",
    "\n",
    "# Let's look at 50 rows of data\n",
    "raw_data.show(50, False)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Apply Proper Schema\n",
    "\n",
    "Let's start with applying the proper schema.\n",
    "\n",
    "From the documentation (see README):\n",
    "> The data is a CSV with emoticons removed. Data file format has 6 fields:  \n",
    "0 - the polarity of the tweet (0 = negative, 2 = neutral, 4 = positive)  \n",
    "1 - the id of the tweet (2087)  \n",
    "2 - the date of the tweet (Sat May 16 23:58:44 UTC 2009)  \n",
    "3 - the query (lyx). If there is no query, then this value is NO_QUERY.  \n",
    "4 - the user that tweeted (robotickilldozr)  \n",
    "5 - the text of the tweet (Lyx is cool)  \n",
    "\n",
    "Let's apply the learnings from our Data Exploration phase and make a proper schema DDL.  \n",
    "We will create a new SparkReader and store it in spark_reader to be used by our next steps."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "schema_ddl = \"\"\"\n",
    "polarity STRING COMMENT \"the polarity of the tweet (0 = negative, 2 = neutral, 4 = positive)\",\n",
    "id LONG COMMENT \"the id of the tweet (2087)\",\n",
    "date TIMESTAMP COMMENT \"the date of the tweet (Sat May 16 23:58:44 UTC 2009)\",\n",
    "query STRING COMMENT \"the query (lyx). If there is no query, then this value is NO_QUERY.\",\n",
    "user string COMMENT \"the user that tweeted (robotickilldozr)\",\n",
    "text string COMMENT \"the text of the tweet (Lyx is cool)\"\n",
    "\"\"\"\n",
    "\n",
    "spark_reader = spark.read.schema(schema_ddl)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Converting Date column\n",
    "\n",
    "Next, we will convert the date column to `TimestampType` by applying `timestampFormat` to our `spark_reader.csv()` method \n",
    "\n",
    "We will use Java SimpleDateFormat: https://docs.oracle.com/javase/8/docs/api/java/text/SimpleDateFormat.html\n",
    "\n",
    "Based on the sample provided (`Sat May 16 23:58:44 UTC 2009`) we can determine that the correct format should be:\n",
    "```java\n",
    "\"EEE MMM dd HH:mm:ss zzz yyyy\"\n",
    "```"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "simple_date_format = \"EEE MMM dd HH:mm:ss zzz yyyy\"\n",
    "\n",
    "raw_data = spark_reader.csv(RAW_PATH, timestampFormat=simple_date_format)\n",
    "raw_data.show()\n",
    "\n",
    "raw_data.summary().show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Cleaning the tweet text\n",
    "\n",
    "We're going to have to do the following to the data in the text column:\n",
    "- Remove email-addresses and URLs\n",
    "- Extract and then remove user-names (@mentions)\n",
    "- Extract and then remove hash-tags (#hash-tag)\n",
    "\n",
    "### Generate some test data"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Let's start with getting some data to test our RegEx on\n",
    "raw_data.select(\"text\").show(50, False)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "It looks like the top 50 rows of data have @-mentions, #-hashtags, and some URLs. No email-addresses are present.  \n",
    "Let's assume that email-addresses also exist in our data, and design accordingly.\n",
    "\n",
    "Assuming this, we can generate a few lines of test data to apply our RegEx on containing some valid and invalid cases, and we can add some email-addresses and different style URLs as wel:\n",
    "```\n",
    "?Obama Administration Must Stop Bonuses to AIG Ponzi Schemers ... http://bit.ly/2CUIg\n",
    "@spinuzzi: Has been a bit crazy, with steep learning curve, but LyX is really good for long docs. For anything shorter, it would be insane.\n",
    "@phyreman9 Google is always a good place to look. Should've mentioned I worked on the Mustang w/ my Dad, @KimbleT.\n",
    "[#MLUC09] Customer Innovation Award Winner: Booz Allen Hamilton -- http://ping.fm/c2hPP\n",
    "#lebron best athlete of our generation, if not all time (basketball related) I don't want to get into inter-sport debates about   __1/2\n",
    "localhost:1234\n",
    "some@email.adress\n",
    "some.email@domain.com\n",
    "a_much_more_complicated_email.some.email@domain.with.lots.of.dots.and_underscores.longdomainname\n",
    "www.something.com\n",
    "http://spark.apache.org/\n",
    "file://some_file_location/file.fmt\n",
    "file:///some_file_location/file.fmt\n",
    "https://blip.fm/~6emhv\n",
    "https://www.freeformatter.com/java-regex-tester.html#ad-output\n",
    "https://spark.apache.org/docs/2.4.3/api/python/pyspark.ml.html#module-pyspark.ml.evaluation\n",
    "google@gmail.com\n",
    "```\n",
    "\n",
    "\n",
    "> __Note__ Spark uses Java style RegEx which is not neccesarily the same as Python.\n",
    "\n",
    "I like to use this website for designing and testing RegEx for Spark.:  \n",
    "https://www.freeformatter.com/java-regex-tester.html\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Extracting/Removing UserNames:\n",
    "\n",
    "Let's start with Usernames\n",
    "\n",
    "According to twitter\n",
    "> Your username cannot be longer than 15 characters. Your real name can be longer (20 characters), but usernames are kept shorter for the sake of ease. A username can only contain alphanumeric characters (letters A-Z, numbers 0-9) with the exception of underscores, as noted above. Check to make sure your desired username doesn't contain any symbols, dashes, or spaces.\n",
    "\n",
    "So now let's design a Regex for doing just this.\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# extract twitter user names/handles to the output column `user_mentioned`\n",
    "\n",
    "user_regex = r\"(@\\w{1,15})\"\n",
    "\n",
    "raw_data.select(\n",
    "    f.regexp_extract(f.col(\"text\"), twitter_mention_regex, 1).alias(\"user_mentioned\"),\n",
    "    \"text\",\n",
    ").show(50, False)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "A limitation that we have here though is that it only extracts the first mention of a user. This presents a limitation of how Spark works with `regexp_extract`.\n",
    "\n",
    "You can see in this Jira Improvement a request to add a `regexp_match_all` method: https://issues.apache.org/jira/browse/SPARK-24884\n",
    "\n",
    "\n",
    "In the interim, we have to run our `regexp_match` multiple times. An example:\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# extract up to 6 twitter user names/handles to the output column `users_mentioned`\n",
    "\n",
    "user_regex = r\"(@\\w{1,15})\"\n",
    "\n",
    "raw_data.select(\n",
    "    f.array_remove(\n",
    "        f.array(\n",
    "            f.regexp_extract(f.col(\"text\"), user_regex, 1),\n",
    "            f.regexp_extract(\n",
    "                f.col(\"text\"), \"\".join([f\"{user_regex}.*?\" for i in range(0, 2)]), 2\n",
    "            ),\n",
    "            f.regexp_extract(\n",
    "                f.col(\"text\"), \"\".join([f\"{user_regex}.*?\" for i in range(0, 3)]), 3\n",
    "            ),\n",
    "            f.regexp_extract(\n",
    "                f.col(\"text\"), \"\".join([f\"{user_regex}.*?\" for i in range(0, 4)]), 4\n",
    "            ),\n",
    "            f.regexp_extract(\n",
    "                f.col(\"text\"), \"\".join([f\"{user_regex}.*?\" for i in range(0, 5)]), 5\n",
    "            ),\n",
    "            f.regexp_extract(\n",
    "                f.col(\"text\"), \"\".join([f\"{user_regex}.*?\" for i in range(0, 6)]), 6\n",
    "            ),\n",
    "        ),\n",
    "        \"\",\n",
    "    ).alias(\"users_mentioned\"),\n",
    "    \"text\",\n",
    ").toPandas().head(35)\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Of course, one could code this into a function, something like this"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def regexp_extract_all(\n",
    "    df: DataFrame,\n",
    "    regex: str,\n",
    "    no_of_extracts: int,\n",
    "    input_column_name: str,\n",
    "    output_column_name: str = \"output\",\n",
    "    empty_array_replace: bool = True,\n",
    "):\n",
    "    \"\"\"Pyspark implementation for extracting all matches of a reg_exp_match\n",
    "    \n",
    "    Parameters\n",
    "    ----------\n",
    "    df: DataFrame\n",
    "        Input DataFrame\n",
    "    \n",
    "    regex: str\n",
    "        Regexp string to extract from input DataFrame\n",
    "    \n",
    "    no_of_extracts: int\n",
    "        Max number of occurrences to extract\n",
    "    \n",
    "    input_column_name: str\n",
    "        Name of the input column\n",
    "    \n",
    "    output_column_name: str\n",
    "        Name of the output column (default: output)\n",
    "    \n",
    "    empty_array_replace: bool\n",
    "        If set to True, will replace empty arrays with null values (default: True)\n",
    "    \"\"\"\n",
    "    repeats = range(0, no_of_extracts)\n",
    "    \n",
    "    # A set of interim columns are created that will be dropped afterwards\n",
    "    match_columns = [f\"___{r}___\" for r in repeats]\n",
    "    \n",
    "    # Apply regexp_extract an r number of times\n",
    "    for r in repeats:\n",
    "        df = df.withColumn(\n",
    "            match_columns[r],\n",
    "            f.regexp_extract(\n",
    "                f.col(input_column_name),\n",
    "                # the input regex string is amended with \".*?\" \n",
    "                # and repeated an r number of times\n",
    "                # r needs to be +1 as matching groups are 1-index\n",
    "                \"\".join([f\"{regex}.*?\" for i in range(0, r+1)]),\n",
    "                r+1,\n",
    "            ),\n",
    "        )\n",
    "\n",
    "    # Create a distinct array, all empty strings removed\n",
    "    df = df.withColumn(\n",
    "        output_column_name,\n",
    "        f.array_remove(f.array_distinct(f.array(match_columns)), \"\"),\n",
    "    )\n",
    "\n",
    "    # Replace empty string with None if empty_array_replace was set \n",
    "    if empty_array_replace:\n",
    "        df = df.withColumn(\n",
    "            output_column_name,\n",
    "            f.when(f.size(output_column_name) == 0, f.lit(None)).otherwise(\n",
    "                f.col(output_column_name)\n",
    "            ),\n",
    "        )\n",
    "    \n",
    "    # Drop interim columns\n",
    "    for c in match_columns:\n",
    "        df = df.drop(c)\n",
    "\n",
    "    return df\n",
    "\n",
    "\n",
    "# Example usage\n",
    "regexp_extract_all(raw_data, user_regex, 6, \"text\", \"users_mentioned\", True).select(\n",
    "    \"text\", \"users_mentioned\"\n",
    ").show(35, False)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "But... we don't just want to extract the mentioned user, we want to also remove it from the text. Let's do that next using `regexp_replace`"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "raw_data.select(\n",
    "    f.regexp_replace(f.col(\"text\"), user_regex, \"\").alias(\"text\"),\n",
    "    f.col(\"text\").alias(\"original_text\"),\n",
    ").toPandas().head(20)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "This little test shows that our RegEx is working. However, we do have one side-effect of this relatively simplistic approach, which is that the `...@domain.` part of email-addresses are also seen as hashtags. For now, I will accept that bug, because we can remove Emails and URLs first. Meaning, we can work around this by applying right order of operation, rather than complicating our RegEx even further.\n",
    "\n",
    "### Extracting hashtags\n",
    "\n",
    "Now let's repeat the process, but this time to extract and remove hashtags.\n",
    "\n",
    "Hashtags follow a very similar approach to @-mentions, only now we use the `#`-character in place of the `@`-symbol.\n",
    "\n",
    "This is what Twitter has to see about hashtags: https://help.twitter.com/en/using-twitter/how-to-use-hashtags  \n",
    "- There is no mention of a maximum length a hashtag can be. \n",
    "\n",
    "So, we can modify our `user_regex` accordingly:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "hashtag_regex = \"(#\\w{1,})\"\n",
    "\n",
    "temp_df = raw_data.filter(f.instr(f.col(\"text\"), \"#\") > 4)\n",
    "\n",
    "# Let's use the function we just created to extract the hashtags\n",
    "_ = regexp_extract_all(temp_df, hashtag_regex, 8, \"text\", \"hashtags\", True).select(\n",
    "    \"text\", \"hashtags\"\n",
    ")\n",
    "\n",
    "_.show(35, False)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Now that we extracted the hashtags, let's convert them to regular words. We start by slightly modifying the Regex, so that the matching group is now outside the hashtag, then we simply apply a `regexp_replace`, where we replace the found hashtag with a version of itself without a hashtag added."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "hashtag_replace_regex = \"#(\\w{1,})\"\n",
    "\n",
    "_.select(f.regexp_replace(f.col(\"text\"), hashtag_replace_regex, \"$1\"), \"hashtags\").show(35, False)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": []
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": []
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": []
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Removing URLs and Emails\n",
    "\n",
    "Now let's repeat the process, but this time to remove URLs and Emails."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "url_regex=r\"((https?|ftp|file):\\/{2,3})+([-\\w+&@#/%=~|$?!:,.]*)|(www.)+([-\\w+&@#/%=~|$?!:,.]*)\"\n",
    "email_regex=r\"[\\w.-]+@[\\w.-]+\\.[a-zA-Z]{1,}\""
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "raw_data.select(\n",
    "    f.regexp_replace(f.col(\"text\"), email_regex, \"\").alias(\"text_no_email\"),\n",
    "    f.regexp_replace(f.col(\"text\"), url_regex, \"\").alias(\"text_no_url\"),\n",
    "    f.col(\"text\").alias(\"original_text\"),\n",
    ").toPandas().head(20)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## HTML Decoding\n",
    "\n",
    "Next up, let's look at the HTML unescaping issue.\n",
    "\n",
    "When looking at the raw text, the HTML does not appear properly unescaped, we're going to have to fix that.  \n",
    "examples of the issue:\n",
    "```\n",
    "started to think that Citi is in really deep s&amp;^t. Are they gonna survive the turmoil or are they gonna be the next AIG?\n",
    "\"I'm listening to \"\"P.Y.T\"\" by Danny Gokey &lt;3 &lt;3 &lt;3 Aww\n",
    "```\n",
    "\n",
    "We can do this by creating a UDF (User Defined Function)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "from pyspark.sql.functions import udf\n",
    "import html\n",
    "\n",
    "@udf\n",
    "def html_unescape(s: str):\n",
    "    return html.unescape(s)\n",
    "\n",
    "raw_data.select(html_unescape(\"text\")).show(35, False)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "As you can see, no more weird HTML escape characters any more :)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": []
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": []
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": []
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": []
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": []
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": []
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": []
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Assemble your cleaning process\n",
    "\n",
    "So now that we have applied our individual fixes, let's assemble our cleaning process"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def regexp_extract_all(\n",
    "    df: DataFrame,\n",
    "    regex: str,\n",
    "    no_of_extracts: int,\n",
    "    input_column_name: str,\n",
    "    output_column_name: str = \"output\",\n",
    "    empty_array_replace: bool = True,\n",
    "):\n",
    "    \"\"\"Pyspark implementation for extracting all matches of a reg_exp_match\n",
    "    \n",
    "    Parameters\n",
    "    ----------\n",
    "    df: DataFrame\n",
    "        Input DataFrame\n",
    "    \n",
    "    regex: str\n",
    "        Regexp string to extract from input DataFrame\n",
    "    \n",
    "    no_of_extracts: int\n",
    "        Max number of occurrences to extract\n",
    "    \n",
    "    input_column_name: str\n",
    "        Name of the input column\n",
    "    \n",
    "    output_column_name: str\n",
    "        Name of the output column (default: output)\n",
    "    \n",
    "    empty_array_replace: bool\n",
    "        If set to True, will replace empty arrays with null values (default: True)\n",
    "    \"\"\"\n",
    "    repeats = range(0, no_of_extracts)\n",
    "\n",
    "    # A set of interim columns are created that will be dropped afterwards\n",
    "    match_columns = [f\"___{r}___\" for r in repeats]\n",
    "\n",
    "    # Apply regexp_extract an r number of times\n",
    "    for r in repeats:\n",
    "        df = df.withColumn(\n",
    "            match_columns[r],\n",
    "            f.regexp_extract(\n",
    "                f.col(input_column_name),\n",
    "                # the input regex string is amended with \".*?\"\n",
    "                # and repeated an r number of times\n",
    "                # r needs to be +1 as matching groups are 1-indexed\n",
    "                \"\".join([f\"{regex}.*?\" for i in range(0, r + 1)]),\n",
    "                r + 1,\n",
    "            ),\n",
    "        )\n",
    "\n",
    "    # Create a distinct array, all empty strings removed\n",
    "    df = df.withColumn(\n",
    "        output_column_name,\n",
    "        f.array_remove(f.array_distinct(f.array(match_columns)), \"\"),\n",
    "    )\n",
    "\n",
    "    # Replace empty string with None if empty_array_replace was set\n",
    "    if empty_array_replace:\n",
    "        df = df.withColumn(\n",
    "            output_column_name,\n",
    "            f.when(f.size(output_column_name) == 0, f.lit(None)).otherwise(\n",
    "                f.col(output_column_name)\n",
    "            ),\n",
    "        )\n",
    "\n",
    "    # Drop interim columns\n",
    "    for c in match_columns:\n",
    "        df = df.drop(c)\n",
    "\n",
    "    return df\n",
    "\n",
    "\n",
    "@udf\n",
    "def html_unescape(s: str):\n",
    "    if isinstance(s, str):\n",
    "        return html.unescape(s)\n",
    "    return s\n",
    "\n",
    "\n",
    "def cleaning_process(df: DataFrame):\n",
    "    url_regex = r\"((https?|ftp|file):\\/{2,3})+([-\\w+&@#/%=~|$?!:,.]*)|(www.)+([-\\w+&@#/%=~|$?!:,.]*)\"\n",
    "    email_regex = r\"[\\w.-]+@[\\w.-]+\\.[a-zA-Z]{1,}\"\n",
    "    user_regex = r\"(@\\w{1,15})\"\n",
    "    hashtag_regex = \"(#\\w{1,})\"\n",
    "    hashtag_replace_regex = \"#(\\w{1,})\"\n",
    "\n",
    "    # Cleaning process:\n",
    "\n",
    "    # 1. Extract user mentions and hashtags\n",
    "    df = regexp_extract_all(df, user_regex, 6, \"text\", \"users_mentioned\", True)\n",
    "    df = regexp_extract_all(df, hashtag_regex, 6, \"text\", \"hashtags\", True)\n",
    "\n",
    "    # 2. Remove email addresses, URLs, and user mentions\n",
    "    # Order is important here\n",
    "    df = (\n",
    "        df\n",
    "        .withColumn(\"original_text\", f.col(\"text\"))\n",
    "        .withColumn(\"text\", f.regexp_replace(f.col(\"text\"), url_regex, \"\"))\n",
    "        .withColumn(\"text\", f.regexp_replace(f.col(\"text\"), email_regex, \"\"))\n",
    "        .withColumn(\"text\", f.regexp_replace(f.col(\"text\"), user_regex, \"\"))\n",
    "    )\n",
    "\n",
    "    # 3. Extract words from hashtags\n",
    "    df = df.withColumn(\n",
    "        \"text\", f.regexp_replace(f.col(\"text\"), hashtag_replace_regex, \"$1\")\n",
    "    )\n",
    "\n",
    "    # 4. Unescape HTML\n",
    "    df = df.withColumn(\"text\", html_unescape(\"text\"))\n",
    "\n",
    "    # 5. Drop all rows with no data in text after cleaning steps were applied\n",
    "    # df = df.filter(f.col(\"text\") != \"\").na.drop(subset=\"text\")\n",
    "    return df\n",
    "\n",
    "schema = \"\"\"\n",
    "polarity STRING COMMENT \"the polarity of the tweet (0 = negative, 2 = neutral, 4 = positive)\",\n",
    "id INT COMMENT \"the id of the tweet (2087)\",\n",
    "date STRING COMMENT \"the date of the tweet (Sat May 16 23:58:44 UTC 2009)\",\n",
    "query STRING COMMENT \"the query (lyx). If there is no query, then this value is NO_QUERY.\",\n",
    "user string COMMENT \"the user that tweeted (robotickilldozr)\",\n",
    "text string COMMENT \"the text of the tweet (Lyx is cool)\"\n",
    "\"\"\"\n",
    "\n",
    "raw_data = spark.read.schema(schema).csv(RAW_PATH)\n",
    "clean_data = cleaning_process(raw_data)\n",
    "clean_data.show()\n",
    "clean_data.select(\"text\").show(250, False)\n",
    "\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "raw_data.count()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "clean_data.count()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "clean_data.filter(\"text == ''\").show(1000)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": []
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": []
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": []
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": []
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": []
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "clean_data.write.parquet(CLEAN_PATH)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": []
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.7.3"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 2
}
